In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Input, BatchNormalization
from tensorflow.keras.applications import VGG19, Xception, InceptionV3, MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

photo_size = 224
batch_size = 64
epochs = 50

def prepare_dataset(data_dir, num_images, subset=None):
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255
    )
    generator = datagen.flow_from_directory(
        data_dir,
        target_size=(photo_size, photo_size),
        class_mode='binary',
        batch_size=batch_size,
        classes=['non_autistic', 'autistic'],
        shuffle=True,
        subset=subset
    )
    return generator

# Function to define and train model with learning rate scheduler
def train_model_with_lr_scheduler(base_model_fn, model_name, train_images, val_images, test_images):
    # Create base model
    base_model = base_model_fn(weights='imagenet', include_top=False, input_shape=(photo_size, photo_size, 3))
    
    # Freeze base model layers
    base_model.trainable = False
    
    # Add kayers for binary classification
    flat = Flatten()(base_model.output)
    dense1 = Dense(256, activation='relu')(flat)
    batch_norm1 = BatchNormalization()(dense1)
    dense2 = Dense(128, activation='relu')(batch_norm1)
    batch_norm2 = BatchNormalization()(dense2)
    output = Dense(1, activation='sigmoid')(batch_norm2)
    
    # Define model
    model = Model(inputs=base_model.input, outputs=output)
    
    # Compile model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Prepare datasets
    train_data = prepare_dataset('C:/Users/meetj/OneDrive/Desktop/ASD/AutismDataset/train', train_images, subset='training')
    validation_data = prepare_dataset('C:/Users/meetj/OneDrive/Desktop/ASD/AutismDataset/valid', val_images)
    test_data = prepare_dataset('C:/Users/meetj/OneDrive/Desktop/ASD/AutismDataset/test', test_images)
    
    # Early Stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    # Learning rate scheduler
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
    
    # Train model with early stopping and learning rate scheduler
    history = model.fit(
        train_data,
        epochs=epochs,
        validation_data=validation_data,
        callbacks=[early_stopping, reduce_lr]
    )
    
    # Evaluate model on test data
    test_loss, test_accuracy = model.evaluate(test_data)
    print("Test Accuracy:", test_accuracy)
    
    # Save model
    model.save("{}.h5".format(model_name))
    
    return model, history

# Number of images in train, validation, and test sets
train_images = 2540
val_images = 300
test_images = 300

# Train VGG19 model with learning rate scheduler
vgg19_model, vgg19_history = train_model_with_lr_scheduler(VGG19, 'vgg19_lr_scheduler', train_images, val_images, test_images)

# Train Xception model with learning rate scheduler
xception_model, xception_history = train_model_with_lr_scheduler(Xception, 'xception_lr_scheduler', train_images, val_images, test_images)

# Train InceptionV3 model with learning rate scheduler
inception_model, inception_history = train_model_with_lr_scheduler(InceptionV3, 'inception_lr_scheduler', train_images, val_images, test_images)

# Train MobileNet model with learning rate scheduler
mobilenet_model, mobilenet_history = train_model_with_lr_scheduler(MobileNet, 'mobilenet_lr_scheduler', train_images, val_images, test_images)

In [ ]:
import matplotlib.pyplot as plt

# Load the training history of the MobileNet model
history = vgg19_history.history

# Plot training and validation accuracy
plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from sklearn.metrics import classification_report

# Load the VGG19 model
vgg19_model = tf.keras.models.load_model("vgg19_lr_scheduler.h5")

# Define image size expected by the model
photo_size = 224

# Function to preprocess the image
def preprocess_image(image_path):
    # Load and resize the image
    img = load_img(image_path, target_size=(photo_size, photo_size))
    # Convert image to array and normalize pixel values
    img_array = img_to_array(img) / 255.0
    # Expand dimensions to match model input shape
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Directory containing images
image_folder = "C:/Users/meetj/OneDrive/Desktop/ASD/AutismDataset/valid"

# Initialize lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through all image files in the folder
for subdir in os.listdir(image_folder):
    if os.path.isdir(os.path.join(image_folder, subdir)):
        for filename in os.listdir(os.path.join(image_folder, subdir)):
            if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
                image_path = os.path.join(image_folder, subdir, filename)
                # Get true label from folder name
                true_label = 1 if subdir == "Autistic" else 0
                true_labels.append(true_label)
                # Preprocess the image
                processed_image = preprocess_image(image_path)
                # Make prediction
                prediction = vgg19_model.predict(processed_image)[0][0]  # Extracting the single prediction value
                # Assign predicted label based on prediction threshold (0.5)
                predicted_label = 1 if prediction >= 0.5 else 0
                predicted_labels.append(predicted_label)

# Convert lists to arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Print classification report
print(classification_report(true_labels, predicted_labels))


In [ ]:
import matplotlib.pyplot as plt

# Load the training history of the MobileNet model
history = xception_history.history

# Plot training and validation accuracy
plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from sklearn.metrics import classification_report

# Load the VGG19 model
vgg19_model = tf.keras.models.load_model("xception_lr_scheduler.h5")

# Define image size expected by the model
photo_size = 224

# Function to preprocess the image
def preprocess_image(image_path):
    # Load and resize the image
    img = load_img(image_path, target_size=(photo_size, photo_size))
    # Convert image to array and normalize pixel values
    img_array = img_to_array(img) / 255.0
    # Expand dimensions to match model input shape
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Directory containing images
image_folder = "C:/Users/meetj/OneDrive/Desktop/ASD/AutismDataset/valid"

# Initialize lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through all image files in the folder
for subdir in os.listdir(image_folder):
    if os.path.isdir(os.path.join(image_folder, subdir)):
        for filename in os.listdir(os.path.join(image_folder, subdir)):
            if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
                image_path = os.path.join(image_folder, subdir, filename)
                # Get true label from folder name
                true_label = 1 if subdir == "Autistic" else 0
                true_labels.append(true_label)
                # Preprocess the image
                processed_image = preprocess_image(image_path)
                # Make prediction
                prediction = vgg19_model.predict(processed_image)[0][0]  # Extracting the single prediction value
                # Assign predicted label based on prediction threshold (0.5)
                predicted_label = 1 if prediction >= 0.5 else 0
                predicted_labels.append(predicted_label)

# Convert lists to arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Print classification report
print(classification_report(true_labels, predicted_labels))


In [ ]:
import matplotlib.pyplot as plt

# Load the training history of the MobileNet model
history = inception_history.history

# Plot training and validation accuracy
plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from sklearn.metrics import classification_report

# Load the VGG19 model
vgg19_model = tf.keras.models.load_model("inception_lr_scheduler.h5")

# Define image size expected by the model
photo_size = 224

# Function to preprocess the image
def preprocess_image(image_path):
    # Load and resize the image
    img = load_img(image_path, target_size=(photo_size, photo_size))
    # Convert image to array and normalize pixel values
    img_array = img_to_array(img) / 255.0
    # Expand dimensions to match model input shape
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Directory containing images
image_folder = "C:/Users/meetj/OneDrive/Desktop/ASD/AutismDataset/valid"

# Initialize lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through all image files in the folder
for subdir in os.listdir(image_folder):
    if os.path.isdir(os.path.join(image_folder, subdir)):
        for filename in os.listdir(os.path.join(image_folder, subdir)):
            if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
                image_path = os.path.join(image_folder, subdir, filename)
                # Get true label from folder name
                true_label = 1 if subdir == "Autistic" else 0
                true_labels.append(true_label)
                # Preprocess the image
                processed_image = preprocess_image(image_path)
                # Make prediction
                prediction = vgg19_model.predict(processed_image)[0][0]  # Extracting the single prediction value
                # Assign predicted label based on prediction threshold (0.5)
                predicted_label = 1 if prediction >= 0.5 else 0
                predicted_labels.append(predicted_label)

# Convert lists to arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Print classification report
print(classification_report(true_labels, predicted_labels))


In [ ]:
import matplotlib.pyplot as plt

# Load the training history of the MobileNet model
history = mobilenet_history.history

# Plot training and validation accuracy
plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from sklearn.metrics import classification_report

# Load the VGG19 model
vgg19_model = tf.keras.models.load_model("mobilenet_lr_scheduler.h5")

# Define image size expected by the model
photo_size = 224

# Function to preprocess the image
def preprocess_image(image_path):
    # Load and resize the image
    img = load_img(image_path, target_size=(photo_size, photo_size))
    # Convert image to array and normalize pixel values
    img_array = img_to_array(img) / 255.0
    # Expand dimensions to match model input shape
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Directory containing images
image_folder = "C:/Users/meetj/OneDrive/Desktop/ASD/AutismDataset/valid"

# Initialize lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through all image files in the folder
for subdir in os.listdir(image_folder):
    if os.path.isdir(os.path.join(image_folder, subdir)):
        for filename in os.listdir(os.path.join(image_folder, subdir)):
            if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
                image_path = os.path.join(image_folder, subdir, filename)
                # Get true label from folder name
                true_label = 1 if subdir == "Autistic" else 0
                true_labels.append(true_label)
                # Preprocess the image
                processed_image = preprocess_image(image_path)
                # Make prediction
                prediction = vgg19_model.predict(processed_image)[0][0]  # Extracting the single prediction value
                # Assign predicted label based on prediction threshold (0.5)
                predicted_label = 1 if prediction >= 0.5 else 0
                predicted_labels.append(predicted_label)

# Convert lists to arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Print classification report
print(classification_report(true_labels, predicted_labels))


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from sklearn.metrics import accuracy_score

# Define image size expected by the models
photo_size = 224

# Load the saved models
vgg19_model = tf.keras.models.load_model("vgg19_lr_scheduler.h5")
mobilenet_model = tf.keras.models.load_model("mobilenet_lr_scheduler.h5")

# Define weights for the models
vgg19_weight = 0.6
mobilenet_weight = 0.4

# Function to preprocess the image
def preprocess_image(image_path):
    # Load and resize the image
    img = load_img(image_path, target_size=(photo_size, photo_size))
    # Convert image to array and normalize pixel values
    img_array = img_to_array(img) / 255.0
    # Expand dimensions to match model input shape
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to make predictions using weighted average
def make_weighted_prediction(image_path, vgg19_model, mobilenet_model, vgg19_weight, mobilenet_weight):
    # Preprocess the image
    processed_image = preprocess_image(image_path)
    # Make predictions using both models
    vgg19_prediction = vgg19_model.predict(processed_image)[0][0]
    mobilenet_prediction = mobilenet_model.predict(processed_image)[0][0]
    # Combine predictions using weighted average
    weighted_prediction = (vgg19_weight * vgg19_prediction) + (mobilenet_weight * mobilenet_prediction)
    return weighted_prediction

# Directory containing images
image_folder = "C:/Users/meetj/OneDrive/Desktop/ASD/AutismDataset/valid"

# Initialize lists to store ground truth labels and predictions
true_labels = []
predictions = []

# Iterate through all image files in the folder
for subdir in os.listdir(image_folder):
    if os.path.isdir(os.path.join(image_folder, subdir)):
        for filename in os.listdir(os.path.join(image_folder, subdir)):
            if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
                image_path = os.path.join(image_folder, subdir, filename)
                # Get ground truth label from folder name
                true_label = 1 if subdir == "Autistic" else 0
                true_labels.append(true_label)
                # Make weighted prediction
                prediction = make_weighted_prediction(image_path, vgg19_model, mobilenet_model, vgg19_weight, mobilenet_weight)
                predictions.append(prediction)

# Convert lists to arrays for accuracy calculation
true_labels = np.array(true_labels)
predictions = np.array(predictions)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions.round())
print("Accuracy:", accuracy)


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define image size expected by the models
photo_size = 224

# Load the saved models
vgg19_model = tf.keras.models.load_model("vgg19_lr_scheduler.h5")
mobilenet_model = tf.keras.models.load_model("mobilenet_lr_scheduler.h5")

# Define weights for the models
vgg19_weight = 0.6
mobilenet_weight = 0.4

# Function to preprocess the image
def preprocess_image(image_path):
    # Load and resize the image
    img = load_img(image_path, target_size=(photo_size, photo_size))
    # Convert image to array and normalize pixel values
    img_array = img_to_array(img) / 255.0
    # Expand dimensions to match model input shape
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to make predictions using weighted average
def make_weighted_prediction(image_path, vgg19_model, mobilenet_model, vgg19_weight, mobilenet_weight):
    # Preprocess the image
    processed_image = preprocess_image(image_path)
    # Make predictions using both models
    vgg19_prediction = vgg19_model.predict(processed_image)[0][0]
    mobilenet_prediction = mobilenet_model.predict(processed_image)[0][0]
    # Combine predictions using weighted average
    weighted_prediction = (vgg19_weight * vgg19_prediction) + (mobilenet_weight * mobilenet_prediction)
    return weighted_prediction

# Define ground truth labels and predictions
true_labels = []
predictions = []

# Directory containing images
image_folder = "C:/Users/meetj/OneDrive/Desktop/ASD/AutismDataset/valid"

# Iterate through all image files in the folder
for subdir in os.listdir(image_folder):
    if os.path.isdir(os.path.join(image_folder, subdir)):
        for filename in os.listdir(os.path.join(image_folder, subdir)):
            if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
                image_path = os.path.join(image_folder, subdir, filename)
                # Get ground truth label from folder name
                true_label = 1 if subdir == "Autistic" else 0
                true_labels.append(true_label)
                # Make weighted prediction
                prediction = make_weighted_prediction(image_path, vgg19_model, mobilenet_model, vgg19_weight, mobilenet_weight)
                predictions.append(prediction)

# Convert lists to arrays for classification report
true_labels = np.array(true_labels)
predictions = np.array(predictions)

# Round predictions to 0 or 1
rounded_predictions = np.round(predictions)

# Generate classification report
report = classification_report(true_labels, rounded_predictions, target_names=['Non_Autistic', 'Autistic'])

# Calculate accuracy and loss
accuracy = np.mean(true_labels == rounded_predictions)
loss = np.mean(tf.keras.losses.binary_crossentropy(true_labels, predictions))

# Print classification report, accuracy, and loss
print(report)
print("Accuracy:", accuracy)
print("Loss:", loss)

# Plotting accuracy
plt.figure(figsize=(10, 5))
plt.plot(true_labels, label='True Labels')
plt.plot(rounded_predictions, label='Predictions')
plt.title('Accuracy: Predictions vs True Labels')
plt.xlabel('Samples')
plt.ylabel('Class')
plt.legend()
plt.show()

# Plotting loss
plt.figure(figsize=(10, 5))
plt.plot(predictions, label='Predictions')
plt.plot(true_labels, label='True Labels')
plt.title('Loss: Predictions vs True Labels')
plt.xlabel('Samples')
plt.ylabel('Probability')
plt.legend()
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Define image size expected by the models
photo_size = 224

# Load the saved models
vgg19_model = tf.keras.models.load_model("vgg19_lr_scheduler.h5")
mobilenet_model = tf.keras.models.load_model("mobilenet_lr_scheduler.h5")

# Define weights for the models
vgg19_weight = 0.6
mobilenet_weight = 0.4

# Function to preprocess the image
def preprocess_image(image_path):
    # Load and resize the image
    img = load_img(image_path, target_size=(photo_size, photo_size))
    # Convert image to array and normalize pixel values
    img_array = img_to_array(img) / 255.0
    # Expand dimensions to match model input shape
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to make predictions using weighted average
def make_weighted_prediction(image_path, vgg19_model, mobilenet_model, vgg19_weight, mobilenet_weight):
    # Preprocess the image
    processed_image = preprocess_image(image_path)
    # Make predictions using both models
    vgg19_prediction = vgg19_model.predict(processed_image)[0][0]
    mobilenet_prediction = mobilenet_model.predict(processed_image)[0][0]
    # Combine predictions using weighted average
    weighted_prediction = (vgg19_weight * vgg19_prediction) + (mobilenet_weight * mobilenet_prediction)
    return weighted_prediction

# Function to interpret the prediction
def interpret_prediction(prediction):
    if prediction >= 0.5:
        return "Autistic"
    else:
        return "Not Autistic"

# Path to the single image you want to predict
image_path = "C:/Users/meetj/OneDrive/Desktop/ASD/Models/Predictions/Autistic.101.jpg"  # Replace with the actual path

# Make prediction for the single image
prediction = make_weighted_prediction(image_path, vgg19_model, mobilenet_model, vgg19_weight, mobilenet_weight)
result = interpret_prediction(prediction)
print("Prediction for", image_path, ":", result)
